Conexão com o banco de dados

In [95]:
import pandas as pd
import psycopg2  

# Configurações de conexão
try:
    conn = psycopg2.connect(
        host="localhost",
        database="projeto_semiestruturados",
        port=5432,
        user="postgres",
        password="@renan123"
    )
    print("Conexão estabelecida com sucesso!")
except psycopg2.Error as e:
    print(f"Erro ao conectar ao PostgreSQL: {e}")
    


Conexão estabelecida com sucesso!


Comunicação com o banco de dados

In [98]:
cursor = conn.cursor()


Leitura do arquivo auxiliar

In [97]:
df = pd.read_csv("base_dados/df_auxiliar.csv", encoding='utf-8')

Pega os dados do arquivo, e coloca-os na Tabela geral do Banco de dados

In [106]:
for _, row in df.iterrows():
    try:
        cursor.execute("""
            INSERT INTO Instituicoes (CODIGO_DA_IES, NOME_DA_IES, SIGLA, CATEGORIA_DA_IES, 
            COMUNITARIA, CONFESSIONAL, FILANTROPICA, ORGANIZACAO_ACADEMICA, SITUACAO_IES) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""", (
            int(row['CODIGO_DA_IES']),
            row['NOME_DA_IES'],
            row['SIGLA'],
            row['CATEGORIA_DA_IES'],
            bool(row['COMUNITARIA']),
            bool(row['CONFESSIONAL']),
            bool(row['FILANTROPICA']),
            row['ORGANIZACAO_ACADEMICA'],
            row['SITUACAO_IES']
        ))
    except Exception as e:
        print(f"Erro na linha {row['CODIGO_DA_IES']}: {e}")
        conn.rollback()  

Teste para ver se foi os dados para o PostgreSQL

In [107]:
cursor.execute("SELECT COUNT(*) FROM Instituicoes")
print("Total de registros:", cursor.fetchone()[0])

Total de registros: 4525


Insere os dados na Tabela Municipio 

In [108]:
df_municipio = df[['CODIGO_DA_IES', 'MUNICIPIO', 'UF']].drop_duplicates()

for _, row in df_municipio.iterrows():
    try:
        cursor.execute("""
            INSERT INTO MUNICIPIO (CODIGO_DA_IES, MUNICIPIO, UF) 
            VALUES (%s, %s, %s)""", (
            str(row['CODIGO_DA_IES']),
            row['MUNICIPIO'],
            row['UF']
        ))
    except Exception as e:
        print(f"Erro na linha {row['CODIGO_DA_IES']}: {e}")
        conn.rollback()

Insere os dados na Tabela Categoria_IES 

In [109]:
conn.rollback()
df_categoria_IES = df[['SIGLA', 'CATEGORIA_DA_IES']].drop_duplicates()


for _, row in df_categoria_IES.iterrows():
    try:
        cursor.execute(""" 
            INSERT INTO CATEGORIA_DA_IES ( SIGLA, CATEGORIA_DA_IES) 
            VALUES (%s, %s)""", (
            row['SIGLA'],
            row['CATEGORIA_DA_IES']
        ))
    except Exception as e:
        print(f"Erro na linha {row['SIGLA']}: {e}")
        conn.rollback()

Insere os dados na Tabela Organizacao_Academcica 

In [ ]:
df_organizacao_academica = df[['ORGANIZACAO_ACADEMICA']].drop_duplicates()

for _, row in df_organizacao_academica.iterrows():
    try:
        cursor.execute("""
            INSERT INTO ORGANIZACAO_ACADEMICA ( ORGANIZACAO_ACADEMICA) 
            VALUES (%s) """, (
            row['ORGANIZACAO_ACADEMICA']
        ))
    except Exception as e:
        print(f"Erro na linha {row['ORGANIZACAO_ACADEMICA']}: {e}")
        conn.rollback()